# 🧬 StrandWeaver PathGNN Training on Google Colab

Train the **PathGNN** (Graph Neural Network for assembly path prediction) on a free T4 GPU.

**What this notebook does:**
1. Installs StrandWeaver + PyTorch + PyTorch Geometric
2. Generates synthetic training data (or loads uploaded data)
3. Converts edge/node CSV → PyG graph `Data` objects
4. Trains PathGNNModel (EdgeConv + GATv2 attention) with early stopping
5. Exports `pathgnn_model.pt` for download

**Runtime:** Set to **GPU** via `Runtime → Change runtime type → T4 GPU`

## 1. Setup & Installation

In [ ]:
# ── Verify GPU availability ──────────────────────────────────────────
import subprocess, sys
result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                        capture_output=True, text=True)
if result.returncode == 0:
    print(f"✓ GPU detected: {result.stdout.strip()}")
else:
    print("⚠ No GPU detected — training will be slower on CPU")

In [ ]:
# ── Install dependencies ─────────────────────────────────────────────
# PyTorch + PyG for the GNN, plus StrandWeaver itself
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q torch-geometric
!pip install -q xgboost scikit-learn

# Clone StrandWeaver and install
!git clone https://github.com/pgrady1322/strandweaver.git /content/strandweaver 2>/dev/null || echo "Already cloned"
!cd /content/strandweaver && pip install -e . --no-deps -q

print("\n✓ Installation complete")

In [ ]:
# ── Verify imports ────────────────────────────────────────────────────
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, MessagePassing
from torch_geometric.data import Data, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch {torch.__version__}  |  Device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}  |  "
          f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

## 2. Upload or Generate Training Data

⚡ **Recommended: Option A (Upload)**  
Data generation is CPU-only and takes **hours** on Colab. Generate locally instead, then upload just the graph CSVs (~50 MB). The GPU is only used during training (Section 5).

**Option A:** Upload pre-generated graph CSVs from your local machine *(~1 min)*  
**Option B:** Generate fresh data on Colab *(~30 min per genome — NOT recommended)*

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# OPTION A: Upload graph CSVs from local machine  (⚡ RECOMMENDED)
# ═══════════════════════════════════════════════════════════════════════
# Step 1 — On your LOCAL machine, package just the graph CSV files:
#
#   cd /path/to/strandweaver      # dev branch
#   find training_data_10x -name "*.csv" -path "*/graph_training/*" \
#       | tar czf graph_csvs.tar.gz -T -
#
# Step 2 — Upload & extract here:

from google.colab import files
uploaded = files.upload()  # Upload graph_csvs.tar.gz

import os, tarfile
with tarfile.open("graph_csvs.tar.gz", "r:gz") as tar:
    tar.extractall("/content/")

# Auto-detect which directory was extracted
DATA_DIR = "/content/training_data_10x" if os.path.isdir("/content/training_data_10x") else "/content/training_data"

# Find all genome directories (handles both flat and batch-nested layouts)
import glob
genome_dirs = sorted(glob.glob(f'{DATA_DIR}/**/genome_*', recursive=True))
if not genome_dirs:
    genome_dirs = sorted(glob.glob(f'{DATA_DIR}/genome_*'))

print(f"✓ Found {len(genome_dirs)} genome directories in {DATA_DIR}")
print(f"  Example: {genome_dirs[0] if genome_dirs else 'NONE'}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# OPTION B: Generate data on Colab  (⚠️ SLOW — NOT recommended)
# ═══════════════════════════════════════════════════════════════════════
# Data generation is CPU-only (~30 min/genome). Only use this if you
# cannot generate locally. Reduce -n to 3-5 for a quick test.
#
# GENERATE_ON_COLAB = False  # Flip to True to enable
# DATA_DIR = '/content/training_data'
#
# if GENERATE_ON_COLAB:
#     !cd /content/strandweaver && python -m strandweaver.cli train generate-data \
#         --genome-size 1000000 -n 5 \
#         --read-types hifi --read-types ont --read-types hic \
#         --coverage 30 --coverage 20 --coverage 15 \
#         --repeat-density 0.35 --graph-training --seed 42 \
#         -o {DATA_DIR}
#     print(f"\n✓ Training data generated in {DATA_DIR}")

## 3. Load Graph Data & Build PyG Datasets

In [ ]:
import csv
import json
import glob
import numpy as np
from pathlib import Path
from collections import defaultdict

# ── Schema v2.0 — Feature columns (must match graph_training_data.py) ──

# Metadata columns prepended to every CSV row (skipped by loader)
METADATA_COLUMNS = [
    'genome_id', 'genome_size', 'chromosome_id', 'read_technology',
    'coverage_depth', 'error_rate', 'ploidy', 'gc_content_global',
    'repeat_density_global', 'heterozygosity_rate', 'random_seed',
    'generator_version', 'schema_version',
]

EDGE_AI_FEATURES = [
    'overlap_length', 'overlap_identity', 'read1_length', 'read2_length',
    'coverage_r1', 'coverage_r2', 'gc_content_r1', 'gc_content_r2',
    'repeat_fraction_r1', 'repeat_fraction_r2',
    'kmer_diversity_r1', 'kmer_diversity_r2',
    'branching_factor_r1', 'branching_factor_r2',
    'hic_support', 'mapping_quality_r1', 'mapping_quality_r2',
    # v2.0: graph topology
    'clustering_coeff_r1', 'clustering_coeff_r2', 'component_size',
    # v2.0: sequence complexity
    'entropy_r1', 'entropy_r2', 'homopolymer_max_r1', 'homopolymer_max_r2',
]

EDGE_AI_PROVENANCE = [
    'node_id_r1', 'node_id_r2',
    'read1_haplotype', 'read2_haplotype',
    'genomic_distance', 'is_repeat_region',
]

NODE_FEATURES = [
    'coverage', 'gc_content', 'repeat_fraction', 'kmer_diversity',
    'branching_factor', 'hic_contact_density', 'allele_frequency',
    'heterozygosity', 'phase_consistency', 'mappability',
    'hic_intra_contacts', 'hic_inter_contacts',
    'hic_contact_ratio', 'hic_phase_signal',
    # v2.0: graph topology
    'clustering_coeff', 'component_size',
    # v2.0: sequence complexity
    'shannon_entropy', 'dinucleotide_bias',
    'homopolymer_max_run', 'homopolymer_density', 'low_complexity_fraction',
    # v2.0: coverage distribution
    'coverage_skewness', 'coverage_kurtosis', 'coverage_cv',
    'coverage_p10', 'coverage_p90',
]

NODE_PROVENANCE = [
    'node_id', 'read_haplotype', 'read_start_pos', 'read_end_pos',
    'read_length', 'is_in_repeat', 'read_technology',
]

# Labels aligned with actual training data output
EDGE_LABEL_MAP = {'TRUE': 0, 'ALLELIC': 1, 'REPEAT': 2, 'SV_BREAK': 3, 'CHIMERIC': 4}
NODE_LABEL_MAP = {'HAP_A': 0, 'HAP_B': 1}
NUM_NODE_CLASSES = 2
NUM_EDGE_CLASSES = 5  # v2.0: added REPEAT class

print(f"Schema v2.0")
print(f"Edge features: {len(EDGE_AI_FEATURES)}")
print(f"Node features: {len(NODE_FEATURES)}")
print(f"Edge classes: {NUM_EDGE_CLASSES}  Node classes: {NUM_NODE_CLASSES}")

In [ ]:
def load_graph_from_csvs(genome_dir: str) -> Data:
    """Load a single genome's graph data into a PyG Data object.

    Schema v2.0: Handles metadata columns (prepended), provenance columns
    (appended before label), and the expanded feature sets. Also backward-
    compatible with v1.0 CSVs (missing features are zero-padded).

    Uses node_id_r1/node_id_r2 columns for REAL graph topology (not random).
    Also loads node_id from diploid CSV for consistent node ordering.

    Args:
        genome_dir: Path to a genome directory (e.g. .../genome_0001).
                    Expects a graph_training/ subdirectory inside.
    """
    graph_dir = Path(genome_dir) / 'graph_training'
    if not graph_dir.exists():
        if Path(genome_dir).name == 'graph_training':
            graph_dir = Path(genome_dir)
        else:
            return None

    # ── Load edge data ────────────────────────────────────────────────
    edge_csv = list(graph_dir.glob('edge_ai_training_*.csv'))
    if not edge_csv:
        return None

    edge_features = []
    edge_labels = []
    edge_node_ids = []  # (source_id, target_id) per edge
    has_node_ids = False

    with open(edge_csv[0]) as f:
        reader = csv.DictReader(f)
        # Check for node ID columns (provenance — v2.0)
        has_node_ids = ('node_id_r1' in reader.fieldnames and
                        'node_id_r2' in reader.fieldnames)

        # Check which features are available (v1 may have fewer)
        available_edge_feats = [c for c in EDGE_AI_FEATURES if c in reader.fieldnames]
        if len(available_edge_feats) < 10:
            print(f"  ⚠ {edge_csv[0].name}: too few features ({len(available_edge_feats)})")
            return None

        for row in reader:
            try:
                # Use available features, zero-pad any missing (v1→v2 compat)
                feat = [float(row.get(c, 0.0)) for c in EDGE_AI_FEATURES]
                lbl = EDGE_LABEL_MAP.get(row['label'], len(EDGE_LABEL_MAP) - 1)
                edge_features.append(feat)
                edge_labels.append(lbl)
                if has_node_ids:
                    edge_node_ids.append((row['node_id_r1'], row['node_id_r2']))
            except (KeyError, ValueError):
                continue

    if len(edge_features) < 10:
        return None

    # ── Load node data ────────────────────────────────────────────────
    node_csv = list(graph_dir.glob('diploid_ai_training_*.csv'))
    node_features = []
    node_labels = []
    node_id_list = []  # ordered node IDs
    has_node_id_col = False

    if node_csv:
        with open(node_csv[0]) as f:
            reader = csv.DictReader(f)
            has_node_id_col = 'node_id' in reader.fieldnames

            # Check which features are available (v1→v2 compat)
            available_node_feats = [c for c in NODE_FEATURES if c in reader.fieldnames]
            if len(available_node_feats) < 5:
                print(f"  ⚠ {node_csv[0].name}: too few matching columns ({len(available_node_feats)})")

            for row in reader:
                try:
                    # Use available features, zero-pad missing ones (v1→v2 compat)
                    feat = [float(row.get(c, 0.0)) for c in NODE_FEATURES]
                    lbl = NODE_LABEL_MAP.get(row.get('haplotype_label', ''), -1)
                    node_features.append(feat)
                    node_labels.append(lbl)
                    if has_node_id_col:
                        node_id_list.append(row['node_id'])
                except (KeyError, ValueError):
                    continue

    num_nodes = max(len(node_features), 100)

    # ── Build edge_index from real topology ───────────────────────────
    num_edges = len(edge_features)

    if has_node_ids and node_id_list:
        # Real topology: map node string IDs → integer indices
        node_id_to_idx = {nid: idx for idx, nid in enumerate(node_id_list)}
        src_list, dst_list = [], []
        keep_mask = []
        for i, (src_id, dst_id) in enumerate(edge_node_ids):
            src_idx = node_id_to_idx.get(src_id)
            dst_idx = node_id_to_idx.get(dst_id)
            if src_idx is not None and dst_idx is not None and src_idx != dst_idx:
                src_list.append(src_idx)
                dst_list.append(dst_idx)
                keep_mask.append(True)
            else:
                keep_mask.append(False)
        src = np.array(src_list, dtype=np.int64)
        dst = np.array(dst_list, dtype=np.int64)
        edge_features = [edge_features[i] for i in range(len(keep_mask)) if keep_mask[i]]
        edge_labels = [edge_labels[i] for i in range(len(keep_mask)) if keep_mask[i]]
    else:
        # Fallback: random topology for old CSVs without node IDs
        src = np.random.RandomState(42).randint(0, num_nodes, size=num_edges)
        dst = np.random.RandomState(43).randint(0, num_nodes, size=num_edges)
        mask = src != dst
        src, dst = src[mask], dst[mask]
        edge_features = [edge_features[i] for i in range(len(mask)) if mask[i]]
        edge_labels = [edge_labels[i] for i in range(len(mask)) if mask[i]]

    # ── Build node features ──────────────────────────────────────────
    if len(node_features) < num_nodes:
        while len(node_features) < num_nodes:
            node_features.append([0.0] * len(NODE_FEATURES))
            node_labels.append(-1)
    elif len(node_features) > num_nodes:
        node_features = node_features[:num_nodes]
        node_labels = node_labels[:num_nodes]

    # ── Convert to tensors ────────────────────────────────────────────
    x = torch.tensor(node_features, dtype=torch.float)
    edge_index = torch.tensor([src.tolist(), dst.tolist()], dtype=torch.long)
    edge_attr = torch.tensor(edge_features, dtype=torch.float)
    y_edge = torch.tensor(edge_labels, dtype=torch.long)
    y_node = torch.tensor(node_labels, dtype=torch.long)

    # Binary edge label: TRUE (0) vs everything else
    y_edge_binary = (y_edge == 0).float()

    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        y_edge=y_edge,
        y_edge_binary=y_edge_binary,
        y_node=y_node,
    )
    return data

print("Graph loader defined ✓ (v2.0 schema, real topology from node IDs)")

In [ ]:
# ── Load all genome graphs ───────────────────────────────────────────
# genome_dirs was set in Cell 7 (upload) or Cell 8 (generate)
print(f"Loading {len(genome_dirs)} genomes...")

all_graphs = []
for i, gd in enumerate(genome_dirs):
    data = load_graph_from_csvs(gd)
    if data is not None:
        all_graphs.append(data)
    if (i + 1) % 50 == 0:
        print(f"  Loaded {i + 1}/{len(genome_dirs)} ({len(all_graphs)} valid)...")

print(f"\n✓ Loaded {len(all_graphs)} graphs from {len(genome_dirs)} directories")
if all_graphs:
    g = all_graphs[0]
    print(f"  Example graph: {g.num_nodes} nodes, {g.num_edges} edges")
    print(f"  Node features: {g.x.shape}")
    print(f"  Edge features: {g.edge_attr.shape}")
    print(f"  Edge label dist: {dict(zip(*np.unique(g.y_edge.numpy(), return_counts=True)))}")
else:
    raise RuntimeError("No valid graphs loaded! Check DATA_DIR and CSV contents.")

In [ ]:
# ── Train/val split ──────────────────────────────────────────────────
from sklearn.model_selection import train_test_split

train_graphs, val_graphs = train_test_split(
    all_graphs, test_size=0.2, random_state=42)

print(f"Training graphs: {len(train_graphs)}")
print(f"Validation graphs: {len(val_graphs)}")

## 4. Define PathGNN Model

In [ ]:
from dataclasses import dataclass
from typing import Dict, Optional

@dataclass
class GNNConfig:
    """Configuration for GNN model.

    v2.0 schema: num_node_features=26 (was 14), num_edge_features=24 (was 17).
    Added graph topology, sequence complexity, and coverage distribution features.
    """
    num_node_features: int = 26   # v2.0: 14 base + 2 topology + 5 complexity + 5 coverage
    num_edge_features: int = 24   # v2.0: 17 base + 3 topology + 4 complexity
    hidden_channels: int = 64
    num_layers: int = 3
    dropout: float = 0.2
    learning_rate: float = 1e-3
    weight_decay: float = 1e-5
    batch_size: int = 1          # graph-level batching
    epochs: int = 100
    patience: int = 15
    use_attention: bool = True
    use_batch_norm: bool = True
    attention_heads: int = 4
    output_dim: int = 1
    num_node_classes: int = NUM_NODE_CLASSES  # HAP_A, HAP_B
    num_ensemble_features: int = 0  # XGBoost ensemble features per edge
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'


class EdgeConvLayer(MessagePassing):
    """Custom edge convolution layer with edge features."""
    def __init__(self, in_channels, out_channels, edge_dim):
        super().__init__(aggr='mean')
        self.lin_src = nn.Linear(in_channels, out_channels)
        self.lin_dst = nn.Linear(in_channels, out_channels)
        self.lin_edge = nn.Linear(edge_dim, out_channels)
        self.lin_combined = nn.Linear(3 * out_channels, out_channels)

    def forward(self, x, edge_index, edge_attr=None):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_i, x_j, edge_attr=None):
        msg_src = self.lin_src(x_i)
        msg_dst = self.lin_dst(x_j)
        if edge_attr is not None:
            msg_edge = self.lin_edge(edge_attr)
        else:
            msg_edge = torch.zeros_like(msg_src)
        msg = torch.cat([msg_src, msg_dst, msg_edge], dim=-1)
        return F.relu(self.lin_combined(msg))


class PathGNNModel(nn.Module):
    """Graph Neural Network for assembly path prediction.

    Architecture:
      - N EdgeConv layers with BatchNorm
      - Optional GATv2 attention layer
      - Edge classification head (TRUE vs non-TRUE)
      - Node classification head (haplotype assignment)

    Supports ensemble mode: additional XGBoost prediction features
    are concatenated to edge_attr before message passing.
    """
    def __init__(self, config: GNNConfig):
        super().__init__()
        self.config = config
        self.device_name = config.device

        # Total edge feature dim (base + ensemble)
        total_edge_dim = config.num_edge_features + config.num_ensemble_features

        # Edge convolution layers
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        in_ch = config.num_node_features
        for _ in range(config.num_layers):
            self.convs.append(EdgeConvLayer(in_ch, config.hidden_channels, total_edge_dim))
            if config.use_batch_norm:
                self.norms.append(nn.BatchNorm1d(config.hidden_channels))
            else:
                self.norms.append(nn.Identity())
            in_ch = config.hidden_channels

        # Attention layer
        if config.use_attention:
            attn_out_per_head = config.hidden_channels // config.attention_heads
            self.attention = GATv2Conv(
                config.hidden_channels, attn_out_per_head,
                heads=config.attention_heads, dropout=config.dropout)
            attn_out = attn_out_per_head * config.attention_heads
            out_ch = config.hidden_channels + attn_out
        else:
            self.attention = None
            out_ch = config.hidden_channels

        # Prediction heads
        self.edge_head = nn.Sequential(
            nn.Linear(out_ch * 2, config.hidden_channels),
            nn.ReLU(), nn.Dropout(config.dropout),
            nn.Linear(config.hidden_channels, config.output_dim),
            nn.Sigmoid())

        self.node_head = nn.Sequential(
            nn.Linear(out_ch, config.hidden_channels),
            nn.ReLU(), nn.Dropout(config.dropout),
            nn.Linear(config.hidden_channels, config.num_node_classes))

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr if hasattr(data, 'edge_attr') else None

        # Concat ensemble features if present
        if hasattr(data, 'ensemble_edge_feat') and data.ensemble_edge_feat is not None:
            if edge_attr is not None:
                edge_attr = torch.cat([edge_attr, data.ensemble_edge_feat], dim=-1)
            else:
                edge_attr = data.ensemble_edge_feat

        # Message passing
        for conv, norm in zip(self.convs, self.norms):
            x = conv(x, edge_index, edge_attr)
            x = norm(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.config.dropout, training=self.training)

        # Attention
        if self.attention is not None:
            x_attn = self.attention(x, edge_index)
            x = torch.cat([x, x_attn], dim=-1)

        # Edge predictions
        src, dst = edge_index
        edge_repr = torch.cat([x[src], x[dst]], dim=-1)
        edge_logits = self.edge_head(edge_repr).squeeze(-1)

        # Node predictions (log_softmax for CrossEntropyLoss)
        node_logits = self.node_head(x)

        return {'edge_logits': edge_logits, 'node_logits': node_logits, 'node_embeddings': x}


# Instantiate with auto-detected feature dims from loaded data
if all_graphs:
    actual_node_dim = all_graphs[0].x.shape[1]
    actual_edge_dim = all_graphs[0].edge_attr.shape[1]
    config = GNNConfig(num_node_features=actual_node_dim, num_edge_features=actual_edge_dim)
    print(f"Auto-detected feature dims from data: node={actual_node_dim}, edge={actual_edge_dim}")
else:
    config = GNNConfig()
    print(f"Using default feature dims: node={config.num_node_features}, edge={config.num_edge_features}")

model = PathGNNModel(config).to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nPathGNNModel: {total_params:,} parameters")
print(f"  hidden_channels={config.hidden_channels}, layers={config.num_layers}")
print(f"  attention_heads={config.attention_heads}, node_classes={config.num_node_classes}")
print(f"  node_features={config.num_node_features}, edge_features={config.num_edge_features}")
print(f"  ensemble_features={config.num_ensemble_features}")
print(f"  Device: {device}")

## 5. Training Loop

In [ ]:
import time
from sklearn.metrics import f1_score, accuracy_score

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Loss functions
edge_loss_fn = nn.BCELoss()   # Binary: TRUE edge vs not
node_loss_fn = nn.CrossEntropyLoss()  # HAP_A vs HAP_B


def train_epoch(model, graphs, optimizer):
    model.train()
    total_loss = 0
    total_edge_correct = 0
    total_edges = 0

    for data in graphs:
        data = data.to(device)
        optimizer.zero_grad()

        out = model(data)

        # Edge loss (binary: is this a TRUE edge?)
        e_loss = edge_loss_fn(out['edge_logits'], data.y_edge_binary.to(device))

        # Node loss (haplotype classification: HAP_A vs HAP_B)
        n_loss = torch.tensor(0.0, device=device)
        if hasattr(data, 'y_node'):
            valid_mask = data.y_node >= 0  # exclude unknown (-1)
            if valid_mask.sum() > 0:
                n_loss = node_loss_fn(
                    out['node_logits'][valid_mask],
                    data.y_node[valid_mask].to(device))

        loss = e_loss + 0.3 * n_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
        preds = (out['edge_logits'] > 0.5).long()
        total_edge_correct += (preds == data.y_edge_binary.long().to(device)).sum().item()
        total_edges += len(data.y_edge_binary)

    avg_loss = total_loss / len(graphs)
    accuracy = total_edge_correct / max(1, total_edges)
    return avg_loss, accuracy


@torch.no_grad()
def evaluate(model, graphs):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    for data in graphs:
        data = data.to(device)
        out = model(data)

        e_loss = edge_loss_fn(out['edge_logits'], data.y_edge_binary.to(device))
        total_loss += e_loss.item()

        preds = (out['edge_logits'] > 0.5).long().cpu().numpy()
        labels = data.y_edge_binary.long().cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)

    avg_loss = total_loss / max(1, len(graphs))
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    return avg_loss, acc, f1


print("Training functions defined ✓")

In [ ]:
# ── Train! ───────────────────────────────────────────────────────────
NUM_EPOCHS = config.epochs
PATIENCE = config.patience

best_val_loss = float('inf')
best_model_state = None
patience_counter = 0
history = {'train_loss': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}

print(f"Training for up to {NUM_EPOCHS} epochs (patience={PATIENCE})")
print("=" * 70)

t_start = time.time()
for epoch in range(1, NUM_EPOCHS + 1):
    train_loss, train_acc = train_epoch(model, train_graphs, optimizer)
    val_loss, val_acc, val_f1 = evaluate(model, val_graphs)
    scheduler.step(val_loss)

    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
        patience_counter = 0
        marker = ' ★'
    else:
        patience_counter += 1
        marker = ''

    if epoch % 5 == 0 or epoch == 1 or marker:
        print(f"Epoch {epoch:3d}  "
              f"train_loss={train_loss:.4f}  "
              f"val_loss={val_loss:.4f}  "
              f"val_acc={val_acc:.4f}  "
              f"val_f1={val_f1:.4f}{marker}")

    if patience_counter >= PATIENCE:
        print(f"\nEarly stopping at epoch {epoch}")
        break

elapsed = time.time() - t_start
print(f"\nTraining complete in {elapsed:.1f}s")
print(f"Best val loss: {best_val_loss:.4f}")

# Restore best model
if best_model_state:
    model.load_state_dict(best_model_state)
    print("Restored best model checkpoint ✓")

## 6. Training Curves

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history['train_loss'], label='Train Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training & Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(history['val_acc'], label='Val Accuracy', linewidth=2, color='green')
ax2.plot(history['val_f1'], label='Val F1', linewidth=2, color='orange')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Score')
ax2.set_title('Validation Metrics')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/content/pathgnn_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved training curves ✓")

## 7. Export Trained Model

In [ ]:
# ── Save in StrandWeaver-compatible format ────────────────────────────
import os

OUTPUT_DIR = '/content/trained_pathgnn'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Save the full model checkpoint
save_path = os.path.join(OUTPUT_DIR, 'pathgnn_model.pt')
torch.save({
    'model_type': 'PathGNNModel',
    'config': {
        'num_node_features': config.num_node_features,
        'num_edge_features': config.num_edge_features,
        'hidden_channels': config.hidden_channels,
        'num_layers': config.num_layers,
        'dropout': config.dropout,
        'use_attention': config.use_attention,
        'use_batch_norm': config.use_batch_norm,
        'output_dim': config.output_dim,
        'num_ensemble_features': config.num_ensemble_features,
    },
    'model_state': model.state_dict(),
    'training_history': history,
    'best_val_loss': best_val_loss,
    'best_val_f1': max(history['val_f1']) if history['val_f1'] else 0,
}, save_path)

# Also save training metadata
meta = {
    'model_type': 'PathGNNModel',
    'num_parameters': sum(p.numel() for p in model.parameters()),
    'num_training_graphs': len(train_graphs),
    'num_validation_graphs': len(val_graphs),
    'best_val_loss': best_val_loss,
    'best_val_f1': max(history['val_f1']) if history['val_f1'] else 0,
    'epochs_trained': len(history['train_loss']),
    'config': vars(config),
    'uses_real_topology': True,
    'uses_hic_features': True,
}
with open(os.path.join(OUTPUT_DIR, 'training_metadata.json'), 'w') as f:
    json.dump(meta, f, indent=2, default=str)

print(f"Model saved to {save_path}")
print(f"Size: {os.path.getsize(save_path) / 1024:.1f} KB")
print(f"Parameters: {meta['num_parameters']:,}")

In [ ]:
# ── Download model files ─────────────────────────────────────────────
from google.colab import files

# Zip the output directory
!cd /content && tar czf pathgnn_trained.tar.gz trained_pathgnn/

print("\nDownloading trained model...")
print("After download, extract and copy to your StrandWeaver model dir:")
print("  tar xzf pathgnn_trained.tar.gz")
print("  cp trained_pathgnn/pathgnn_model.pt trained_models/pathgnn/pathgnn_model.pt")

files.download('/content/pathgnn_trained.tar.gz')

## 8. Quick Validation

Verify the exported model loads correctly in StrandWeaver's format.

In [ ]:
# ── Verify the model loads ────────────────────────────────────────────
checkpoint = torch.load(save_path, map_location='cpu', weights_only=False)
print("Checkpoint keys:", list(checkpoint.keys()))
print(f"Config: {checkpoint['config']}")
print(f"State dict keys: {len(checkpoint['model_state'])} tensors")
print(f"Best val loss: {checkpoint['best_val_loss']:.4f}")
print(f"Best val F1: {checkpoint['best_val_f1']:.4f}")

# Reload into a fresh model
cfg2 = GNNConfig(**{k: v for k, v in checkpoint['config'].items() if k != 'device'})
model2 = PathGNNModel(cfg2)
model2.load_state_dict(checkpoint['model_state'])
model2.eval()
print(f"\n✓ Model reloads and is ready for inference ({sum(p.numel() for p in model2.parameters()):,} params)")

## 9. Ensemble Mode: XGBoost → GNN Features

Upload pre-trained XGBoost edge models (from the XGBoost Retraining notebook)
to inject their prediction probabilities as extra edge features for the GNN.
This commonly gives 1-3% absolute improvement over either model alone.

In [ ]:
# ── Ensemble: inject XGBoost predictions as GNN edge features ─────────
# Upload edgewarden_hifi.pkl (or any tech's edge model) from the
# XGBoost retraining notebook's output.
#
# Skip this cell if you don't have a pre-trained XGBoost model yet —
# the base GNN will still work fine without ensemble features.

import pickle
from sklearn.preprocessing import StandardScaler

ENSEMBLE_MODE = False  # Set True after uploading XGBoost model

if ENSEMBLE_MODE:
    from google.colab import files as colab_files
    print("Upload edgewarden_hifi.pkl and scaler_hifi.pkl:")
    uploaded_xgb = colab_files.upload()

    xgb_model = pickle.loads(list(uploaded_xgb.values())[0])

    # For each graph, predict edge scores and attach as ensemble features
    for data in all_graphs:
        edge_feat_np = data.edge_attr.numpy()
        # XGBoost predict_proba → N_classes columns (e.g., 4 for edge_ai)
        xgb_probs = xgb_model.predict_proba(edge_feat_np)
        data.ensemble_edge_feat = torch.tensor(xgb_probs, dtype=torch.float)

    NUM_ENSEMBLE = xgb_probs.shape[1]
    print(f"✓ Ensemble features added: {NUM_ENSEMBLE} XGBoost probability columns per edge")
    print("  Re-instantiate the model with num_ensemble_features set, then re-train")
else:
    NUM_ENSEMBLE = 0
    print("Ensemble mode OFF — set ENSEMBLE_MODE = True after uploading XGBoost model")
    print("(The base GNN trains fine without this — ensemble is an optional boost)")

## 10. Hyperparameter Sweep

Quick grid search over the most impactful GNN hyperparameters.
Each combination trains for up to 60 epochs with early stopping.
~2 min per config on T4 GPU → 12 combos ≈ 25 min.

In [ ]:
# ── Hyperparameter sweep ──────────────────────────────────────────────
import itertools

SWEEP_GRID = {
    'hidden_channels': [64, 128],
    'num_layers': [3, 4],
    'dropout': [0.1, 0.3],
    'learning_rate': [1e-3, 5e-4],
}

# Generate all combos
keys = list(SWEEP_GRID.keys())
combos = list(itertools.product(*[SWEEP_GRID[k] for k in keys]))
print(f"Sweeping {len(combos)} hyperparameter combinations...")
print(f"Grid: {SWEEP_GRID}\n")

sweep_results = []

for i, vals in enumerate(combos):
    params = dict(zip(keys, vals))
    print(f"[{i+1}/{len(combos)}] {params}")

    # Build config
    cfg = GNNConfig(
        num_node_features=len(NODE_FEATURES),
        num_edge_features=len(EDGE_AI_FEATURES),
        hidden_channels=params['hidden_channels'],
        num_layers=params['num_layers'],
        dropout=params['dropout'],
        learning_rate=params['learning_rate'],
        epochs=60,       # shorter for sweep
        patience=10,
        num_ensemble_features=NUM_ENSEMBLE,
    )

    # Build and train model
    sweep_model = PathGNNModel(cfg).to(device)
    sweep_opt = torch.optim.Adam(
        sweep_model.parameters(), lr=cfg.learning_rate, weight_decay=cfg.weight_decay)
    sweep_sched = torch.optim.lr_scheduler.ReduceLROnPlateau(
        sweep_opt, mode='min', factor=0.5, patience=3, verbose=False)

    best_vl = float('inf')
    best_vf1 = 0.0
    patience_ctr = 0
    t0 = time.time()

    for epoch in range(1, cfg.epochs + 1):
        tl, ta = train_epoch(sweep_model, train_graphs, sweep_opt)
        vl, va, vf1 = evaluate(sweep_model, val_graphs)
        sweep_sched.step(vl)

        if vl < best_vl:
            best_vl = vl
            best_vf1 = vf1
            patience_ctr = 0
        else:
            patience_ctr += 1

        if patience_ctr >= cfg.patience:
            break

    elapsed = time.time() - t0
    n_params = sum(p.numel() for p in sweep_model.parameters())

    result = {
        **params,
        'best_val_loss': round(best_vl, 4),
        'best_val_f1': round(best_vf1, 4),
        'epochs': epoch,
        'time_s': round(elapsed, 1),
        'params': n_params,
    }
    sweep_results.append(result)
    print(f"  → val_f1={best_vf1:.4f}  val_loss={best_vl:.4f}  "
          f"({epoch} epochs, {elapsed:.1f}s, {n_params:,} params)")

# Sort by val_f1
sweep_results.sort(key=lambda r: r['best_val_f1'], reverse=True)
print(f"\n{'='*80}")
print("TOP 5 CONFIGURATIONS:")
print(f"{'='*80}")
for i, r in enumerate(sweep_results[:5]):
    print(f"  #{i+1}  F1={r['best_val_f1']:.4f}  loss={r['best_val_loss']:.4f}  "
          f"h={r['hidden_channels']} L={r['num_layers']} "
          f"d={r['dropout']} lr={r['learning_rate']}  "
          f"({r['params']:,} params, {r['time_s']:.0f}s)")

# Save sweep results
with open('/content/sweep_results.json', 'w') as f:
    json.dump(sweep_results, f, indent=2)

In [ ]:
# ── Retrain best config for full epochs and export ────────────────────
best = sweep_results[0]
print(f"Retraining best config: h={best['hidden_channels']} L={best['num_layers']} "
      f"d={best['dropout']} lr={best['learning_rate']}")

best_cfg = GNNConfig(
    num_node_features=len(NODE_FEATURES),
    num_edge_features=len(EDGE_AI_FEATURES),
    hidden_channels=best['hidden_channels'],
    num_layers=best['num_layers'],
    dropout=best['dropout'],
    learning_rate=best['learning_rate'],
    epochs=100,
    patience=15,
    num_ensemble_features=NUM_ENSEMBLE,
)

best_model = PathGNNModel(best_cfg).to(device)
best_opt = torch.optim.Adam(
    best_model.parameters(), lr=best_cfg.learning_rate, weight_decay=best_cfg.weight_decay)
best_sched = torch.optim.lr_scheduler.ReduceLROnPlateau(
    best_opt, mode='min', factor=0.5, patience=5, verbose=True)

best_vl = float('inf')
best_state = None
patience_ctr = 0
best_history = {'train_loss': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}

t0 = time.time()
for epoch in range(1, best_cfg.epochs + 1):
    tl, ta = train_epoch(best_model, train_graphs, best_opt)
    vl, va, vf1 = evaluate(best_model, val_graphs)
    best_sched.step(vl)

    best_history['train_loss'].append(tl)
    best_history['val_loss'].append(vl)
    best_history['val_acc'].append(va)
    best_history['val_f1'].append(vf1)

    marker = ''
    if vl < best_vl:
        best_vl = vl
        best_state = best_model.state_dict().copy()
        patience_ctr = 0
        marker = ' ★'
    else:
        patience_ctr += 1

    if epoch % 5 == 0 or epoch == 1 or marker:
        print(f"Epoch {epoch:3d}  train_loss={tl:.4f}  val_loss={vl:.4f}  "
              f"val_acc={va:.4f}  val_f1={vf1:.4f}{marker}")

    if patience_ctr >= best_cfg.patience:
        print(f"\nEarly stopping at epoch {epoch}")
        break

elapsed = time.time() - t0
best_model.load_state_dict(best_state)
print(f"\n✓ Best model retrained in {elapsed:.1f}s")
print(f"  Best val_f1: {max(best_history['val_f1']):.4f}")
print(f"  Best val_loss: {best_vl:.4f}")

# ── Export ────────────────────────────────────────────────────────────
SWEEP_DIR = '/content/trained_pathgnn_sweep'
os.makedirs(SWEEP_DIR, exist_ok=True)
sweep_save = os.path.join(SWEEP_DIR, 'pathgnn_model.pt')
torch.save({
    'model_type': 'PathGNNModel',
    'config': {
        'num_node_features': best_cfg.num_node_features,
        'num_edge_features': best_cfg.num_edge_features,
        'hidden_channels': best_cfg.hidden_channels,
        'num_layers': best_cfg.num_layers,
        'dropout': best_cfg.dropout,
        'use_attention': best_cfg.use_attention,
        'use_batch_norm': best_cfg.use_batch_norm,
        'output_dim': best_cfg.output_dim,
        'num_ensemble_features': best_cfg.num_ensemble_features,
    },
    'model_state': best_model.state_dict(),
    'training_history': best_history,
    'best_val_loss': best_vl,
    'best_val_f1': max(best_history['val_f1']),
    'sweep_results': sweep_results,
}, sweep_save)

meta = {
    'model_type': 'PathGNNModel_sweep',
    'num_parameters': sum(p.numel() for p in best_model.parameters()),
    'best_config': {k: v for k, v in best.items() if k not in ('params', 'time_s')},
    'sweep_combos_tested': len(sweep_results),
    'best_val_f1': max(best_history['val_f1']),
    'best_val_loss': best_vl,
    'uses_real_topology': True,
    'uses_hic_features': True,
    'uses_ensemble': NUM_ENSEMBLE > 0,
}
with open(os.path.join(SWEEP_DIR, 'training_metadata.json'), 'w') as f:
    json.dump(meta, f, indent=2, default=str)

print(f"\nSaved to {sweep_save}")

# Download
from google.colab import files
!cd /content && tar czf pathgnn_sweep.tar.gz trained_pathgnn_sweep/
files.download('/content/pathgnn_sweep.tar.gz')